# Validation notebook of 4-VSC system (GridFollowing): Stepss vs Dynawo
# RMS simulation results for different scenarios

## <center> Simulation setup </center>
| | Stepss | Dynawo | EMTP
|-| :---: | :---: | :---: |
| Domain | RMS | RMS | EMT |
| Duration | 10s | 10s | 10s |
| Time step | 1ms (fix) | 10us-10ms (variable) & 1ms (fix) | 50us (fix) |
| Solver solution method | DAE | DAE | Nodal |
| Solver integration method | Backward Differentiation Formula (order2) | BDF(IDA) & BE, Trapezoidal | Trapezoidal |

## Plotting Function

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib widget

In [ ]:
def autoscale_y(ax,margin):
    """This function rescales the y-axis based on the data that is visible given the current xlim of the axis.
    ax -- a matplotlib axes object
    margin -- the fraction of the total height of the y-data to pad the upper and lower ylims"""

    def get_bottom_top(line):
        xd = line.get_xdata()
        yd = line.get_ydata()
        lo,hi = ax.get_xlim()
        y_displayed = yd[((xd>lo) & (xd<hi))]
        h = np.max(y_displayed) - np.min(y_displayed)
        bot = np.min(y_displayed)-margin*h
        top = np.max(y_displayed)+margin*h
        return bot,top

    lines = ax.get_lines()
    bot,top = np.inf, -np.inf

    for line in lines:
        new_bot, new_top = get_bottom_top(line)
        if new_bot < bot: bot = new_bot
        if new_top > top: top = new_top

    ax.set_ylim(bot,top)

In [ ]:
def plot_results(scenario, stepss_file, dynawo_file, emtp_file):

    stepss = pd.read_csv(stepss_file, sep = '\s+', comment=";", keep_default_na=False, names=('time', 'BUS_A', 'BUS_B', 'BUS_C', 'BUS_E', 'BUS_F'))
    dynawo= pd.read_csv(dynawo_file, sep = ';')
    
    emtp= pd.read_csv(emtp_file, sep = '\t', low_memory=False)
    emtp.drop(index=emtp.index[0], axis=0, inplace=True)
    emtp.rename(columns=lambda x: x.strip(), inplace=True)

    stepss_variables= ['BUS_A', 'BUS_B', 'BUS_E', 'BUS_F']
    dynawo_variables= ['NETWORK__BUS__A_TN_Upu_value', 'NETWORK__BUS__B_TN_Upu_value', 'NETWORK__BUS__E_TN_Upu_value', 'NETWORK__BUS__F_TN_Upu_value']
    # dynawo_variables= ['GFLVSC_HVDC1_injectorGFL_UConvPu', 'GFLVSC_HVDC2_injectorGFL_UConvPu', 'GFLVSC_WP1_injectorGFL_UConvPu', 'GFLVSC_WP2_injectorGFL_UConvPu']
    emtp_variables= ['uamp_pcc_A_pu', 'uamp_pcc_B_pu', 'uamp_pcc_E_pu',	'uamp_pcc_F_pu']
    # emtp_variables= ['uamp_conv_A_pu', 'uamp_conv_B_pu', 'uamp_conv_E_pu',	'uamp_conv_F_pu']
    labels= ['Bus A Vpcc (p.u)', 'Bus B Vpcc (p.u)', 'Bus E Vpcc (p.u)', 'Bus F Vpcc (p.u)']
    
    plt.close('all')
    fig=plt.figure(figsize=(15, 12))
    plt.suptitle(scenario , fontsize=16)
    plt.subplots_adjust(top=0.9, hspace=0.4, wspace=0.4)
    plt.tight_layout()
    
    for n, (x, y, z, l) in enumerate(zip(stepss_variables, dynawo_variables, emtp_variables, labels)):
        ax = plt.subplot(2, 2, n + 1)
        ax.plot(stepss['time'], stepss[x], color= 'black', linestyle='-', label='Stepss')
        ax.plot(dynawo['time'], dynawo[y], color= 'tab:green', linestyle=':', label='Dynawo')
        ax.plot(pd.to_numeric(emtp['X axis'][::10]), pd.to_numeric(emtp[z][::10]), color= 'blue', linestyle='-', label='EMTP')
               
        ax.set_title(l, fontsize=14)
        ax.set_ylabel(l, fontsize=14)
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.tick_params(axis='both', which='minor', labelsize=14)
        ax.legend(fontsize=14, loc="lower right")
        ax.legend(fontsize=14, loc="upper right")
        ax.set_xlim([3, 8])

        autoscale_y(ax, 1)
        
        # if ax.get_title() in ["P (MW)", "Q (Mvar)"]:
        #     autoscale_y(ax, 0.1)
        # else:
        #     autoscale_y(ax, 0.1)

## Operating Point 1

### Event: Mild disturbance: line disconnection A-B Line 2

In [ ]:
plot_results('Scenario 1: Mild disturbance in Operating point 1: line disconnection A-B Line 2',
             './results/4VSC/OP1_DisconnectLine/stepss.csv',
             './results/4VSC/OP1_DisconnectLine/dynawo.csv',
             './results/4VSC/OP1_DisconnectLine/emtp.csv')

### Event: Severe disturbance: fault Bus A (t=100ms)

In [ ]:
plot_results('Scenario 2: Severe disturbance in Operating point 1: fault (t=100ms) in Bus A',
             './results/4VSC/OP1_Fault/stepss.csv', #fault impedance: X=0.001 p.u. (1.6 Ohm)
             './results/4VSC/OP1_Fault/dynawo.csv', #fault impedance: X=0.001 p.u.
             './results/4VSC/OP1_Fault/emtp.csv') #fault impedance: X=0.001 p.u.

## Operating Point 2

### Event: Mild disturbance: line disconnection A-B Line 2

In [ ]:
plot_results('Scenario 3: Mild disturbance in Operating point 2: line disconnection A-B Line 2',
             './results/4VSC/OP2_DisconnectLine/stepss.csv',
             './results/4VSC/OP2_DisconnectLine/dynawo.csv',
             './results/4VSC/OP2_DisconnectLine/emtp.csv')

### Event: Severe disturbance: fault Bus A (t=100ms)

In [ ]:
plot_results('Scenario 4: Severe disturbance in Operating point 2: fault (t=100ms) in Bus A',
             './results/4VSC/OP2_Fault/stepss.csv',  #fault impedance: X=0.001 p.u.
             './results/4VSC/OP2_Fault/dynawo.csv',  #fault impedance: X=0.001 p.u.
             './results/4VSC/OP2_Fault/emtp.csv')